In [36]:
import os
file_path = 'D:/Data/probe_match'

In [37]:
import numpy as np
import pandas as pd

In [38]:
#probe = pd.read_csv(os.path.join(file_path, 'Partition6467ProbePoints.csv'), names = ['sampleID', 'dateTime', 'sourceCode', 'latitude', 'longitude', 'altitude', 'speed', 'heading'])
probe = pd.read_csv(os.path.join(file_path, 'Transformed_ProbePoints.csv'), names = ['sampleID', 'dateTime', 'sourceCode', 'latitude', 'longitude', 'altitude', 'speed', 'heading', 'X', 'Y'], header = 0)
print("probe shape: ", probe.shape)

probe shape:  (3375745, 10)


In [39]:
probe.head()

,sampleID,dateTime,sourceCode,latitude,longitude,altitude,speed,heading,X,Y
0,3496,6/12/2009 6:12:49 AM,13,51.496868,9.386022,200,23,339,-77285.189075,55811.256229
1,3496,6/12/2009 6:12:54 AM,13,51.496682,9.386157,200,10,129,-77276.134716,55790.442706
2,3496,6/12/2009 6:12:59 AM,13,51.496705,9.386422,201,21,60,-77257.710385,55792.679445
3,3496,6/12/2009 6:13:04 AM,13,51.496749,9.386840,201,0,360,-77228.660753,55797.178015
4,3496,6/12/2009 6:13:09 AM,13,51.496864,9.387294,199,0,360,-77196.969435,55809.445141


In [40]:
# convert df into list
id_count = 0
probe_list = []

for id in probe['sampleID'].unique():
    
    probe_list.append(probe[probe['sampleID'] == id][['longitude', 'latitude', 'altitude', 'speed', 'heading']].values.tolist())
          
    id_count += 1
    if id_count >= 100:
        break
        
print(id_count)

100


In [42]:
# build trajectory
test = probe_list[99]
print(test[:5])

[[8.45239254646003, 51.893609426915596, 136.0, 132.0, 54.0], [8.45450562424958, 51.894508218392716, 137.0, 128.0, 56.0], [8.45676429569721, 51.8953820317984, 137.0, 134.0, 59.0], [8.45918473787606, 51.896224245429, 137.0, 139.0, 62.0], [8.4954188670963, 51.903797462582595, 148.0, 142.0, 71.0]]


In [47]:
# trajectory visulization function

import gmaps
import ipywidgets as widgets
gmaps.configure()
def coordMaker(traj):
    lst_lat = [float(i[1]) for i in traj]
    lst_lon = [float(i[0]) for i in traj]
    return (sum(lst_lat)/len(lst_lat), sum(lst_lon)/len(lst_lon))
def traj_to_df(traj):
    labels = ['longitude', 'latitude', 'altitude', 'speed', 'heading']
    return pd.DataFrame.from_records(traj, columns=labels)
def draw_traj(traj):
    if len(traj) > 50:
        zoom = 13
    else:
        zoom = 14
    center_coordinates = coordMaker(traj)
    fig = gmaps.figure(center=center_coordinates, zoom_level=zoom, map_type='TERRAIN')
    probe_layer = gmaps.symbol_layer(
        traj_to_df(traj)[['latitude', 'longitude']], fill_color='red', stroke_color='red', scale=2
    )
    fig.add_layer(probe_layer)
    return fig
draw_traj(test)

Figure(layout=FigureLayout(height='420px'))

In [45]:
# match the reference point
## precise
test

[[8.45239254646003, 51.893609426915596, 136.0, 132.0, 54.0],
 [8.45450562424958, 51.894508218392716, 137.0, 128.0, 56.0],
 [8.45676429569721, 51.8953820317984, 137.0, 134.0, 59.0],
 [8.45918473787606, 51.896224245429, 137.0, 139.0, 62.0],
 [8.4954188670963, 51.903797462582595, 148.0, 142.0, 71.0],
 [8.53787739761174, 51.93957326933739, 166.0, 135.0, 33.0],
 [8.53976542130113, 51.9411653280258, 168.0, 134.0, 39.0],
 [8.542037755250929, 51.9426794350147, 170.0, 142.0, 46.0],
 [8.543769456446169, 51.943633547052706, 171.0, 142.0, 50.0]]

In [ ]:
%%time
# build nodes list with XY coordinates
link = pd.read_csv(os.path.join(file_path, 'Partition6467LinkData.csv'), names = ['linkPVID', 'refNodeID', 'nrefNodeID', 'length', 'functionalClass', 'directionOfTravel', 'speedCategory', 'fromRefSpeedLimit', 'toRefSpeedLimit', 'fromRefNumLanes', 'toRefNumLanes', 'multiDigitized', 'urban', 'timeZone', 'shapeInfo', 'curvatureInfo', 'slopeInfo'])
def nan_split(text, deli):
    import math
    try:
        r = text.split(deli)
    except:
        r = []
    return r
link['shapeInfo'] = link['shapeInfo'].map(lambda x: list(map(lambda y: nan_split(y, '/'), nan_split(x, '|'))))
link['curvatureInfo'] = link['curvatureInfo'].map(lambda x: list(map(lambda y: nan_split(y, '/'), nan_split(x, '|'))))
link['slopeInfo'] = link['slopeInfo'].map(lambda x: list(map(lambda y: nan_split(y, '/'), nan_split(x, '|'))))
nodes = []
nodes_highway = []
nodes_local = []
nodes_ref = []
nodes_nref = []
def addAllNodeList(alink):
    somenodes = alink.tolist()[-3]
    nodes_all = list([somenodes[i][0], somenodes[i][1], somenodes[i][2], i, alink[0]] for i in range(len(somenodes)))
    nodes_ref = [somenodes[0][0], somenodes[0][1], somenodes[0][2], alink[0], alink[1]]
    nodes_nref = [somenodes[-1][0], somenodes[-1][1], somenodes[-1][2], alink[0], alink[2]]
    category = alink[4]
    return [nodes_all, nodes_ref, nodes_nref, category]
for index, row in link.iterrows():
    [v_nodes_all, v_nodes_ref, v_nodes_nref, v_category] = addAllNodeList(row)
    nodes.extend(v_nodes_all)
    nodes_ref.append(v_nodes_ref)
    nodes_nref.append(v_nodes_nref)
    if v_category < 5:
        nodes_highway.extend(v_nodes_all)
    else:
        nodes_local.extend(v_nodes_all)
    #if index > 5000:
    #    break
nodes = pd.DataFrame(nodes, columns = ['latitude', 'longitude', 'elevation', 'sequence', 'link'])
nodes_highway = pd.DataFrame(nodes_highway, columns = ['latitude', 'longitude', 'elevation', 'sequence', 'link'])
nodes_local = pd.DataFrame(nodes_local, columns = ['latitude', 'longitude', 'elevation', 'sequence', 'link'])
nodes_ref = pd.DataFrame(nodes_ref, columns = ['latitude', 'longitude', 'elevation', 'link', 'refNodeID'])
nodes_nref = pd.DataFrame(nodes_nref, columns = ['latitude', 'longitude', 'elevation', 'link', 'nrefNodeID'])

In [ ]:
def nearestNodes(x, y, length):
    return nodes[nodes['X'] > (x - length)
                 & nodes['X'] < (x + length)
                 & nodes['Y'] > (y - length)
                 & nodes['Y'] < (y + length)]['link'].unique()
